In [87]:
from IPython.display import HTML

HTML('''
<script>
  function code_toggle() {    if (code_shown){      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {$('div.input').show('500');$('#toggleButton').val('Hide Code')    }code_shown = !code_shown
  }  $( document ).ready(function(){code_shown=false;$('div.input').hide()
  });</script><form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>
''')

In [88]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import pandas as pd
import re

import os
import json

In [89]:
def strip_exp_names(e, iden='FINAL_RESULTS'):
    i = e.find(iden)+len(iden)+1
    return e[i:-5]

def correct_df_columns(df):
    for row in df.iterrows():
        s = row[0].split('_')
        if s[0] == 'transfer':
    #         df.loc[row[0], 'Net'] = s[3]
            df.loc[row[0], 'Type'] = 'Transfer'
            df.loc[row[0], 'Source Adversary'] = s[4]
            df.loc[row[0], 'Target Adversary'] = s[5]
            df.loc[row[0], 'Transfer'] = s[1] + ' ' + s[2]
        else:
    #         df.loc[row[0], 'Net'] = s[0]
            df.loc[row[0], 'Type'] = 'Baseline'
            df.loc[row[0], 'Source Adversary'] = s[2] if len(s)==3 else 'nat'
            df.loc[row[0], 'Target Adversary'] = ''
            df.loc[row[0], 'Transfer'] = 'No transfer'

In [90]:
model_path = '../attack_results/white_box/'
all_nets =  list(os.listdir(model_path))
all_nets = [a for a in all_nets if 'FINAL_RESULTS' in a]
# all_nets = [strip_exp_names(a) for a in all_nets]
all_nets_cifar10 = [a for a in all_nets if 'cifar100' not in a]

In [116]:
nets = all_nets_cifar10
attack_results = {}

for net in nets:
    fname = model_path+net
    with open(fname,'r+') as json_data:
        attack_results[strip_exp_names(net)] = list(json.load(json_data).values())
        
df = pd.DataFrame.from_dict(attack_results, columns=['Clean', 
                                                     'FGSM 0.03125', 'PGD 0.03125',
                                                     'FGSM 0.0625', 'PGD 0.0625',
                                                    ], orient='index')
            
correct_df_columns(df)
df[df.select_dtypes(include=['number']).columns] *= 100
df=np.round(df,3)

In [127]:
idx = ['Type', 'Transfer', 'Source Adversary', 'Target Adversary']
df = df.sort_values(idx)
df.set_index(idx)#[['Clean', 'FGSM 0.0625','PGD 0.0625']]
# df

Clean  \
Type     Transfer       Source Adversary Target Adversary          
Baseline No transfer    fgsm                               87.12   
                        nat                                92.53   
                        pgd                                83.90   
Transfer 12 layers      fgsm             fgsm              87.21   
                                         nat               89.09   
                        nat              fgsm              77.31   
                                         nat               91.24   
                                         pgd               60.81   
                        pgd              nat               84.63   
                                         pgd               81.26   
         all layers     fgsm             fgsm              85.71   
                                         nat               91.95   
                        nat              fgsm              88.10   
                                         nat               92.65   
                                         pgd               81.53   
                        pgd              nat               91.39   
                                         pgd               82.59   
         feat extractor fgsm             fgsm              66.56   
                                         nat               68.97   
                        nat              fgsm              69.34   
                                         nat               73.10   
                                         pgd               52.15   
                        pgd              nat               64.87   
                                         pgd               61.27   

                                                           FGSM 0.03125  \
Type     Transfer       Source Adversary Target Adversary                 
Baseline No transfer    fgsm                                      84.42   
                        nat                                       25.16   
                        pgd                                       62.76   
Transfer 12 layers      fgsm             fgsm                     81.63   
                                         nat                      55.28   
                        nat              fgsm                     44.14   
                                         nat                      22.53   
                                         pgd                      22.73   
                        pgd              nat                      44.47   
                                         pgd                      62.62   
         all layers     fgsm             fgsm                     78.22   
                                         nat                      27.87   
                        nat              fgsm                     81.50   
                                         nat                      23.62   
                                         pgd                      63.75   
                        pgd              nat                      25.02   
                                         pgd                      65.06   
         feat extractor fgsm             fgsm                     58.78   
                                         nat                      33.16   
                        nat              fgsm                     11.45   
                                         nat                       5.95   
                                         pgd                      13.44   
                        pgd              nat                      35.60   
                                         pgd                      40.79   

                                                           PGD 0.03125  \
Type     Transfer       Source Adversary Target Adversary                
Baseline No transfer    fgsm                                     19.99   
                        nat                                       0.08   
                        pgd                

# Black box attacks

In [128]:
black= json.load(open("../attack_results/black_box/black_box_FINAL_RESULTS_ResNet56.json"))

In [129]:
df_black = pd.DataFrame.from_dict(attack_results, columns=['Clean', 
                                                     'FGSM 0.03125', 'PGD 0.03125',
                                                     'FGSM 0.06250', 'PGD 0.06250',
                                                    ], orient='index')

# for col in df_black.select_dtypes(include=['number']).columns:
#     df_black[col].values[:] = -1 

In [125]:

for eps, adv in black.items():
#     print(adv)
    for adv_res in adv:
        for row_full, res in adv_res.items():
#             print(res)
            row = row_full[:row_full.find('_attacked')]
            if 'attacked_by_fgsm' in row_full:
                adversary = 'fgsm'
            elif 'attacked_by_pgd' in row_full:
                adversary = 'pgd'
            epsilon = eps.split('_')[-1]
            column = adversary.upper() + ' ' + epsilon
#             print(row_full, )
            df_black.loc[row, column] = res

correct_df_columns(df_black)
df_black[df_black.select_dtypes(include=['number']).columns] *= 100
df_black=np.round(df_black,3)

In [126]:
df_black = df_black.drop([a for a in df_black.index if 'cifar100' in a])
idx = ['Type', 'Transfer', 'Source Adversary', 'Target Adversary']
df_black = df_black.sort_values(idx)
df_black.set_index(idx)#[['Clean', 'FGSM 0.0625','PGD 0.0625']]

Clean  \
Type     Transfer       Source Adversary Target Adversary          
Baseline No transfer    fgsm                               87.12   
                        nat                                92.53   
                        pgd                                83.90   
Transfer 12 layers      fgsm             fgsm              87.21   
                                         nat               89.09   
                        nat              fgsm              77.31   
                                         nat               91.24   
                                         pgd               60.81   
                        pgd              nat               84.63   
                                         pgd               81.26   
         all layers     fgsm             fgsm              85.71   
                                         nat               91.95   
                        nat              fgsm              88.10   
                                         nat               92.65   
                                         pgd               81.53   
                        pgd              nat               91.39   
                                         pgd               82.59   
         feat extractor fgsm             fgsm              66.56   
                                         nat               68.97   
                        nat              fgsm              69.34   
                                         nat               73.10   
                                         pgd               52.15   
                        pgd              nat               64.87   
                                         pgd               61.27   

                                                           FGSM 0.03125  \
Type     Transfer       Source Adversary Target Adversary                 
Baseline No transfer    fgsm                                      85.08   
                        nat                                       25.16   
                        pgd                                       82.69   
Transfer 12 layers      fgsm             fgsm                     82.24   
                                         nat                      77.18   
                        nat              fgsm                     70.76   
                                         nat                      62.09   
                                         pgd                      43.64   
                        pgd              nat                      82.73   
                                         pgd                      80.41   
         all layers     fgsm             fgsm                     77.49   
                                         nat                      62.60   
                        nat              fgsm                     79.52   
                                         nat                      58.47   
                                         pgd                      80.46   
                        pgd              nat                      65.36   
                                         pgd                      81.61   
         feat extractor fgsm             fgsm                     63.98   
                                         nat                      60.77   
                        nat              fgsm                     54.57   
                                         nat                      50.44   
                                         pgd                      40.93   
                        pgd              nat                      64.17   
                                         pgd                      60.71   

                                                           PGD 0.03125  \
Type     Transfer       Source Adversary Target Adversary                
Baseline No transfer    fgsm                                     84.80   
                        nat                                       0.08   
                        pgd                